# Predict from Video

In [1]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import os
from utils.frames import count_frames
from utils.plot import create_plot

# Config

In [2]:
LABELS = ["safe", "collision", "tailgating", "weaving"]
FRAMES_PER_SEQ = 30
FRAME_INTERVAL = 1 # todo: implement intervals here
AVG_SIZE = 1

VIDEO_PATH = "input/3.mp4"
MODEL_PATH = "model"
OUTPUT_PATH = "output/v3.mp4"

# Initialize

In [3]:
# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(os.path.join(MODEL_PATH, "model.keras"))
lb = pickle.loads(open(os.path.join(MODEL_PATH, "labels.pickle"), "rb").read())

# predictions queue
Q = deque(maxlen=AVG_SIZE)

# initialize the video stream
vs = cv2.VideoCapture(VIDEO_PATH)
writer = None
(W, H) = (None, None)

# initialize stats variables
num_frames = count_frames(VIDEO_PATH)
prediction_counts = { label: [0] for label in LABELS }
current_frame = 0
max_y = 0
# shape = (FRAMES_PER_SEQ, 224, 224, 3)
sequence = []
label = ""

[INFO] loading model and label binarizer...


# Main Loop

In [4]:
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    
    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed:
        break
    
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    
    # increment the current frame
    current_frame += 1
        
    # process and add the frame to the sequence buffer
    output = frame.copy() # secure a copy before modifying the frame to prepare it for prediction
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Is this really necessary??
    frame = cv2.resize(frame, (224, 224))  # Resize frame if necessary
    frame = np.array(frame)
    sequence.append(frame)
    
    # If we have enough frames, append the sequence to the data and labels
    if len(sequence) == FRAMES_PER_SEQ:  # Assuming N frames per sequence
        
        # make predictions on the frame and then update the predictions queue
        to_predict = np.array(sequence).reshape(FRAMES_PER_SEQ, -1)
        preds = model.predict(np.expand_dims(to_predict, axis=0))[0]
        Q.append(preds)

        # reset sequence buffer for next frame
        sequence = []

        # perform prediction averaging over the current history of previous predictions
        results = np.array(Q).mean(axis=0)
        i = np.argmax(results)
        label = lb.classes_[i]

        print("PREDICTION: ", label)

   # draw the prediction on the output frame
    default_color = (100, 100, 100)
    red = (0, 0, 255)
    green = (0, 255, 0)
    text_x = W - 200
    text_y = H // 2 - 80
    cv2.putText(output, "SAFE", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1,
             green if label == "safe" else default_color, 5)
    cv2.putText(output, "COLLISION", (text_x, text_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 1,
             red if label == "collision" else default_color, 5)
    cv2.putText(output, "TAILGATING", (text_x, text_y + 100), cv2.FONT_HERSHEY_SIMPLEX, 1,
             red if label == "tailgating" else default_color, 5)
    cv2.putText(output, "WEAVING", (text_x, text_y + 150), cv2.FONT_HERSHEY_SIMPLEX, 1,
             red if label == "weaving" else default_color, 5)

    # initialize video writer if not already init
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, 30,
            (W, H), True)
    
    # write the output frame to disk
    writer.write(output)
    	    

    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
	
	# # increment the prediction count
	# for l in lb.classes_:
	# 	new_count = prediction_counts[l][current_frame - 1] + 1 if label == l else prediction_counts[l][current_frame - 1]
	# 	if (new_count > max_y):
	# 		max_y = new_count
	# 	prediction_counts[l].append(new_count)

 
	
	# # draw the matplotlib plot on the output frame
	# plot_img = create_plot(current_frame + 1, max_y + 30, [
	# 	{
	# 		"y_vals": prediction_counts["safe"],
	# 		"label": "safe"
	# 	},
	# 	{
	# 		"y_vals": prediction_counts["collision"],
	# 		"label": "collision"
	# 	},
	# 	{
	# 		"y_vals": prediction_counts["tailgating"],
	# 		"label": "tailgating"
	# 	},
	# 	{
	# 		"y_vals": prediction_counts["weaving"],
	# 		"label": "weaving"
	# 	}
	# ])

	# # resize the plot image to fit the output frame, and convert the color profile
	# plot_img = cv2.resize(plot_img, (W // 3, H // 3))
	# plot_img = cv2.cvtColor(plot_img, cv2.COLOR_RGB2BGR)

	# # add plot to the bottom right of the frame
	# output[H - plot_img.shape[0]:, W - plot_img.shape[1]:W] = plot_img
	

		

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
PREDICTION:  collision
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
PREDICTION:  collision
1/1 ━━━━

# Cleanup

In [5]:
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...
